<a href="https://colab.research.google.com/github/sibyl-dev/Explingo/blob/main/basic_llm_setup.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>